In [1]:
import pandas as pd
import sys
import os
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))
os.chdir(root_dir)

import functions

#Datasets
pET = pd.read_pickle('results/pET_complete.pkl.gz', compression='infer')



from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np

In [2]:
#cross validation sets
def compute_auc(df, labels):
    preds = df['Flexibility'].values
    fpr, tpr, _ = functions.roc_curve(labels, preds)
    return functions.auc(fpr, tpr)



In [3]:
pET['Analysed_seq'] = pET['Protein'].apply(ProteinAnalysis)
pET['Flexibility_scores'] = pET['Analysed_seq'].apply(lambda x:x.flexibility())

#Averaged flexibility
pET['Flexibility'] = pET['Flexibility_scores'].apply(np.mean)

In [4]:
pET['Accession'] = pET.Accession.apply(lambda x:x.split('_')[0])
pET

Accession                                           Sequence  \
pET15 2      BbCD00584211  ATGGGCCATCACCATCACCATCACAGCCATATGATTTTTGTAACTA...   
      3      BcCD00331746  ATGGGCCATCACCATCACCATCACAGCCATATGGATAAAGAGAATC...   
      4      BcCD00331747  ATGGGCCATCACCATCACCATCACAGCCATATGGCACATACTACTA...   
      5      BcCD00341383  ATGGGCCATCACCATCACCATCACAGCCATATGGATAAAGAGAATC...   
      6      BcCD00591009  ATGGGCCATCACCATCACCATCACAGCCATATGAGTTTGAAGGGGA...   
...                   ...                                                ...   
pET21 11432  YsCD00591076  ATGGAGATCTACGTTTTCACCCCGGAAGGGCGCATTGTGGAATTAC...   
      11433  YsCD00591078  ATGGAGATCTACGTTTTCACCCCGGAAGGGCGCATTGTGGAATTAC...   
      11434  YsCD00591079  ATGGATTTATTCCCTGATGAGATCTACGTTTTCACCCCGGAAGGGC...   
      11435  YsCD00665646  ATGGTAGAGCAGGAGCAAGAAGCGATAACTTTTGAAGTGGTAGCCA...   
      11436  YsCD00665990  ATGGAGTTGGGGCTTCGCACTTACTCTGCACGCTTGTTAGGGTCGA...   

             Class  Solubility  \
pET15 2          2           1   
      3          1           0   
      4          2           1   
      5          1           0   
      6          2           1   
...            ...         ...   
pET21 11432      2           1   
      11433      2           1   
      11434      2           1   
      11435      2           1   
      11436      1           0   

                                                       Protein  \
pET15 2      MGHHHHHHSHMIFVTKLNGDGYYLNPYHIESIEANPDTTILLMNGK...   
      3      MGHHHHHHSHMDKENPRVGDKYITVQKVGKKIFEAEVEILEYDAPH...   
      4      MGHHHHHHSHMAHTTTSMEIFGSPEQVWQLIGGFNSLPDWLPYIPS...   
      5      MGHHHHHHSHMDKENPRVGDKYITVQKVGKKIFEAEVEILEYDAPH...   
      6      MGHHHHHHSHMSLKGKRIGFGFTGSHCTYEEVMPHLEKLIAEGAEV...   
...                                                        ...   
pET21 11432  MEIYVFTPEGRIVELPAGATPVDFAYVVHTDIGHACVGARVDRQPY...   
      11433  MEIYVFTPEGRIVELPAGATPVDFAYVVHTDIGHACVGARVDRQPY...   
      11434  MDLFPDEIYVFTPEGRIVELPAGATPVDFAYVVHTDIGHACVGARV...   
      11435  MVEQEQEAITFEVVAREWHASNRQWSEAHSARVLKSLEDNLFQAIG...   
      11436  MELGLRTYSARLLGSNPKLVLHGGGNTSVKLNMNDLFGNPVQVMCI...   

                                                  Analysed_seq  \
pET15 2      <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      3      <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      4      <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      5      <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      6      <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
...                                                        ...   
pET21 11432  <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      11433  <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      11434  <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      11435  <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   
      11436  <Bio.SeqUtils.ProtParam.ProteinAnalysis object...   

                                            Flexibility_scores  Flexibility  
pET15 2      [0.9614166666666667, 0.961857142857143, 0.9615...     1.001985  
      3      [0.9614166666666667, 0.961857142857143, 0.9615...     1.004373  
      4      [0.9614166666666667, 0.961857142857143, 0.9615...     0.999729  
      5      [0.9614166666666667, 0.961857142857143, 0.9615...     1.003355  
      6      [0.9614166666666667, 0.961857142857143, 0.9615...     1.000593  
...                                                        ...          ...  
pET21 11432  [0.9647261904761903, 0.9933690476190475, 1.000...     0.993638  
      11433  [0.9647261904761903, 0.9933690476190475, 1.000...     0.993719  
      11434  [1.0076904761904761, 1.009845238095238, 0.9627...     0.993403  
      11435  [1.0280357142857144, 1.0478214285714287, 1.005...     1.000704  
      11436  [1.0012857142857141, 0.9862738095238096, 0.973...     0.992594  

[12216 rows x 8 columns]

In [5]:
cv_10 = pd.read_pickle('results/testing_10_with_tag.pkl')
cv_40 = pd.read_pickle('results/testing_40_with_tag.pkl')
cv_90 = pd.read_pickle('results/testing_90_with_tag.pkl')



In [6]:

all_aucs_test_flex_10 = []
for i, v in enumerate(cv_10):
    v['Accession'] = v.Accession.apply(lambda x:x.split('_')[0])
    v.merge(pET, on="Accession")
    temp = v.merge(pET, on="Accession")[['Flexibility']].copy()
    labels = v.merge(pET, on="Accession")['Solubility_x'].values
    all_aucs_test_flex_10.append(compute_auc(temp, labels))
all_aucs_test_flex_10

[0.6643632758883635,
 0.6687641696393183,
 0.6969155426238243,
 0.6513430557958907,
 0.683880010595091,
 0.6433989568573065,
 0.7025386688407274,
 0.6813160221508678,
 0.6566436407964242,
 0.6746017453306759]

In [8]:
functions.np.mean(all_aucs_test_flex_10), functions.np.std(all_aucs_test_flex_10)

(0.6723765088518491, 0.018264736250053384)

In [7]:

all_aucs_test_flex_40 = []
for i, v in enumerate(cv_40):
    v['Accession'] = v.Accession.apply(lambda x:x.split('_')[0])
    v.merge(pET, on="Accession")
    temp = v.merge(pET, on="Accession")[['Flexibility']].copy()
    labels = v.merge(pET, on="Accession")['Solubility_x'].values
    all_aucs_test_flex_40.append(compute_auc(temp, labels))
all_aucs_test_flex_40

[0.6560427467189762,
 0.6887902744630072,
 0.6886100896483764,
 0.6529225085259568,
 0.6525980723072307,
 0.6895379563354627,
 0.6771816716555377,
 0.6992528431631844,
 0.6565538440976979,
 0.6690002871821054]

In [7]:
functions.np.mean(all_aucs_test_flex_40), functions.np.std(all_aucs_test_flex_40)

(0.6730490294097535, 0.01693089567108872)

In [8]:

all_aucs_test_flex_90 = []
for i, v in enumerate(cv_90):
    v['Accession'] = v.Accession.apply(lambda x:x.split('_')[0])
    v.merge(pET, on="Accession")
    temp = v.merge(pET, on="Accession")[['Flexibility']].copy()
    labels = v.merge(pET, on="Accession")['Solubility_x'].values
    all_aucs_test_flex_90.append(compute_auc(temp, labels))
all_aucs_test_flex_90

[0.7061846361731474,
 0.6580118238468062,
 0.7016063184757672,
 0.6878643940416987,
 0.6954966697928755,
 0.6279010025062657,
 0.6529156105026542,
 0.6853843861282404,
 0.6786002999357281,
 0.6365827395091053]

In [9]:
functions.np.mean(all_aucs_test_flex_90), functions.np.std(all_aucs_test_flex_90)

(0.673054788091229, 0.026114101211708606)